In [1]:
import pandas as pd
import csv
import datetime
from sqlalchemy import create_engine

In [2]:
# create a reference to the csv and import into pandas dataframe
csv_path = (r"C:\\Users\\abrow\\Desktop\\EMERSON_BOOTCAMP\\BOOTCAMP_AMB\\Project 2\\Project-2-ETL\\DataSets\steam.csv")
steam_games_df = pd.read_csv(csv_path)

In [3]:
#displaying the dataframe
steam_games_df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,11/1/2000,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,4/1/1999,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,5/1/2003,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,6/1/2001,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,11/1/1999,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [4]:
#getting the data types for each column
steam_games_df.dtypes

appid                 int64
name                 object
release_date         object
english               int64
developer            object
publisher            object
platforms            object
required_age          int64
categories           object
genres               object
steamspy_tags        object
achievements          int64
positive_ratings      int64
negative_ratings      int64
average_playtime      int64
median_playtime       int64
owners               object
price               float64
dtype: object

In [5]:
#converting release_date to datetime from object
steam_games_df['release_date']=pd.to_datetime(steam_games_df['release_date'],  infer_datetime_format=True)

In [6]:
#verifying that data type has changed
steam_games_df.dtypes

appid                        int64
name                        object
release_date        datetime64[ns]
english                      int64
developer                   object
publisher                   object
platforms                   object
required_age                 int64
categories                  object
genres                      object
steamspy_tags               object
achievements                 int64
positive_ratings             int64
negative_ratings             int64
average_playtime             int64
median_playtime              int64
owners                      object
price                      float64
dtype: object

In [7]:
# extracting year from 'release_date and moving it to it's own column for joining with other dataframes in 
steam_games_df["release_year"]=pd.DatetimeIndex(steam_games_df['release_date']).year

In [8]:
#verifying that the new column was created and properly displays the data
steam_games_df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,release_year
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,2000
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99,1999
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99,2003
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99,2001
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99,1999


In [11]:
#extracting the columns to comprise our new dataframe to be exported to our SQL database
steam_games_df = steam_games_df[['name', 'platforms', 'release_year']]
steam_games_df.head()

,name,platforms,release_year
0,Counter-Strike,windows;mac;linux,2000
1,Team Fortress Classic,windows;mac;linux,1999
2,Day of Defeat,windows;mac;linux,2003
3,Deathmatch Classic,windows;mac;linux,2001
4,Half-Life: Opposing Force,windows;mac;linux,1999


In [14]:
#renaming the columns to be compatible with SQL database
steam_games_df.columns = ['game', 'platform', 'release_year']
steam_games_df.head()

,game,platform,release_year
0,Counter-Strike,windows;mac;linux,2000
1,Team Fortress Classic,windows;mac;linux,1999
2,Day of Defeat,windows;mac;linux,2003
3,Deathmatch Classic,windows;mac;linux,2001
4,Half-Life: Opposing Force,windows;mac;linux,1999


In [17]:
#creating the connection to the server
rds_connection_string = "postgres:93AImm!2020@localhost:5432/gaming_DB"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [18]:
engine.table_names()

<ipython-input-18-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['clean_allgames', 'combined_games']

In [20]:
#sending the data to the database
steam_games_df.to_sql(name='combined_games', con=engine, if_exists='append', index=False)